<a href="https://colab.research.google.com/github/chychur/Rainforcement_Learning/blob/main/reinforcement_learning(main_part).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rainforsment Learning

We will use the [surprise library](https://surpriselib.com/ "surprise library"), which is an add-on to the [scikit-learn library](https://scikit-learn.org/stable/index.html "scikit-learn library") for training recommender system models.

Let's build a matrix vectorization model based on the movielens dataset (SVD). We will select the best parameters using cross-validation, and compare other algorithms (SVD++, NMF) and determine the optimal one.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162662 sha256=ba62ff7bfdca2453db54e2ff891bc42f0b9b4c23ec7372a7a32fe420262f3ca5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [7]:
import pandas as pd
from surprise import accuracy, Dataset, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate


In [6]:
data = Dataset.load_builtin(name = 'ml-100k' , prompt = True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [8]:
benchmark = []
algorithms = [SVD(), SVDpp(), NMF()]

for algorithm in algorithms:
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = pd.concat([pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']), tmp])
    benchmark.append(tmp)

In [9]:

surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [10]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.917733,28.849804,4.921889
SVD,0.934038,1.754026,0.173988
NMF,0.962043,2.326921,0.193748


In [11]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = SVDpp()
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)

RMSE: 0.9163


0.9162785294787977